# [E-15] 프로젝트: Movielens 영화 추천 실습

<br>

---

***Note!!***

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.


- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.


- 별점을 **시청횟수**로 해석해서 생각하겠습니다.


- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

<br>

## Step 0. 프로젝트에 필요한 주요 라이브러리 버전 확인 및 import
---

In [488]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [489]:
import pandas as pd
import os

from tqdm import tqdm

<br>

## Step 1. 데이터 준비, 전처리 및 탐색
---

- Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음.

#### (1) 데이터 준비
---

In [490]:
rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
rating_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=rating_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


<br>

#### (2) 데이터 전처리
---

In [491]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [492]:
# ratings 컬럼의 이름을 counts로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [493]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [494]:
ratings.index=list(range(len(ratings)))
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
836473,6040,1090,3,956715518
836474,6040,1094,5,956704887
836475,6040,562,5,956704746
836476,6040,1096,4,956715648


<br>

---

***영화 제목을 보기 위해 메타 데이터를 읽어옵니다.***

---

<br>

In [495]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

In [496]:
# 문자열을 소문자로 만들어 준다.

movies['title'] = movies['title'].str.lower()  # 검색을 쉽게 하기 위해 title 문자열을 소문자로 변환
movies['genre'] = movies['genre'].str.lower()  # 검색을 쉽게 하기 위해 genre 문자열을 소문자로 변환
movies.sample(5)

,movie_id,title,genre
3025,3094,maurice (1987),drama|romance
1791,1860,character (karakter) (1997),drama
2944,3013,bride of re-animator (1990),comedy|horror
534,538,six degrees of separation (1993),drama
2012,2081,"little mermaid, the (1989)",animation|children's|comedy|musical|romance


---

<br>

<br>

***ratings에 메타데이터 movies >> title / genre 열 추가하기***

<br>

In [433]:
# movie_title / movie_genre list 만들기

movie_title = []
movie_genre = []

for i in tqdm(range(len(ratings))):
    if ratings.loc[i, 'movie_id'] in movies['movie_id']:
        row_idx = movies.index[movies['movie_id']==ratings.loc[i, 'movie_id']].tolist()
        movie_title.append(movies.loc[row_idx[0], 'title'])
        movie_genre.append(movies.loc[row_idx[0], 'genre'])
    else:
        movie_title.append(np.nan)
        movie_genre.append(np.nan)

100%|██████████| 836478/836478 [02:04<00:00, 6719.83it/s]


In [497]:
print(f'ratings 갯수 : {len(ratings)}')
print(f'movie_title 갯수 : {len(movie_title)}')
print(f'movie_genre 갯수 : {len(movie_genre)}')

ratings 갯수 : 836478
movie_title 갯수 : 836478
movie_genre 갯수 : 836478


In [498]:
ratings['title'] = movie_title
ratings['genre'] = movie_genre

In [499]:
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,1,661,3,978302109,james and the giant peach (1996),animation|children's|musical
2,1,914,3,978301968,my fair lady (1964),musical|romance
3,1,3408,4,978300275,erin brockovich (2000),drama
4,1,2355,5,978824291,"bug's life, a (1998)",animation|children's|comedy


In [500]:
# 열 순서 재배치하기

ratings = ratings[['user_id', 'movie_id', 'title', 'genre', 'counts', 'timestamp']]
ratings.head()

,user_id,movie_id,title,genre,counts,timestamp
0,1,1193,one flew over the cuckoo's nest (1975),drama,5,978300760
1,1,661,james and the giant peach (1996),animation|children's|musical,3,978302109
2,1,914,my fair lady (1964),musical|romance,3,978301968
3,1,3408,erin brockovich (2000),drama,4,978300275
4,1,2355,"bug's life, a (1998)",animation|children's|comedy,5,978824291


<br>

***새로운 column을 추가하고 ratings 데이터프레임 결측치 확인 후 제거***

<br>

In [501]:
ratings.isnull().sum()

user_id         0
movie_id        0
title        7171
genre        7171
counts          0
timestamp       0
dtype: int64

In [502]:
print(f'결측치 제거 전 ratings 갯수 : {len(ratings)}')

ratings.dropna(axis=0, how='any', inplace=True)
print(f'결측치 제거 후 ratings 갯수 : {len(ratings)}')

결측치 제거 전 ratings 갯수 : 836478
결측치 제거 후 ratings 갯수 : 829307


/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [503]:
ratings.index=list(range(len(ratings)))
ratings

,user_id,movie_id,title,genre,counts,timestamp
0,1,1193,one flew over the cuckoo's nest (1975),drama,5,978300760
1,1,661,james and the giant peach (1996),animation|children's|musical,3,978302109
2,1,914,my fair lady (1964),musical|romance,3,978301968
3,1,3408,erin brockovich (2000),drama,4,978300275
4,1,2355,"bug's life, a (1998)",animation|children's|comedy,5,978824291
...,...,...,...,...,...,...
829302,6040,1090,platoon (1986),drama|war,3,956715518
829303,6040,1094,"crying game, the (1992)",drama|romance|war,5,956704887
829304,6040,562,welcome to the dollhouse (1995),comedy|drama,5,956704746
829305,6040,1096,sophie's choice (1982),drama,4,956715648


In [504]:
# 첫번째 유저가 어떤 영화를 시청하는지 확인해 보자.

condition = (ratings['user_id']==ratings.loc[0, 'user_id'])
ratings.loc[condition].sort_values(by='counts', ascending=False).head(20)

,user_id,movie_id,title,genre,counts,timestamp
0,1,1193,one flew over the cuckoo's nest (1975),drama,5,978300760
46,1,1029,dumbo (1941),animation|children's|musical,5,978302205
40,1,1,toy story (1995),animation|children's|comedy,5,978824268
18,1,3105,awakenings (1990),drama,5,978301713
41,1,1961,rain man (1988),drama,5,978301590
23,1,527,schindler's list (1993),drama|war,5,978824195
37,1,1022,cinderella (1950),animation|children's|musical,5,978300055
14,1,1035,"sound of music, the (1965)",musical,5,978301753
25,1,48,pocahontas (1995),animation|children's|musical|romance,5,978824351
45,1,1028,mary poppins (1964),children's|comedy|musical,5,978301777


<br>

#### (3) 데이터 탐색
---

***pandas.DataFrame.nunique() 매서드를 사용하면 특정 컬럼에 포함된 유니크한 데이터의 갯수를 알아보는데 유용하다.***

In [505]:
# rating에 있는 유니크한 영화 개수

ratings['movie_id'].nunique()

3561

In [506]:
# rating에 있는 유니크한 사용자 수

ratings['user_id'].nunique()

6039

In [507]:
# 가장 인기 있는 영화 30개(인기순)

movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [508]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계

user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      137.325219
std       155.064242
min         1.000000
25%        37.000000
50%        80.000000
75%       175.000000
max      1942.000000
Name: movie_id, dtype: float64

In [509]:
# 유저별 시청횟수 중앙값에 대한 통계

user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.056052
std         0.433137
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

<br>

#### (4) 모델 검증을 위한 사용자 초기 정보 셋팅
---

In [510]:
sample_data = ratings.sample(5)
sample_data

,user_id,movie_id,title,genre,counts,timestamp
490346,3622,1097,e.t. the extra-terrestrial (1982),children's|drama|fantasy|sci-fi,4,966570635
655752,4750,3684,"fabulous baker boys, the (1989)",drama|romance,3,963534000
396022,2964,3148,"cider house rules, the (1999)",drama,3,971120805
279761,2015,1213,goodfellas (1990),crime|drama,4,1020387314
617895,4472,3100,"river runs through it, a (1992)",drama,4,965067173


In [511]:
# my_favorite = ['amadeus (1984)' , 'Jumanji (1995)' ,'Gladiator (2000)' ,'Big Daddy (1999)' ,'Alaska (1996)']

# 'jy'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['jy']*5, 'movie_id': sample_data['movie_id'],
                            'title': sample_data['title'], 'genre' : sample_data['genre'],
                            'counts':[5]*5, 'timestamp':sample_data['timestamp']})

if not ratings.isin({'user_id':['jy']})['user_id'].any():  # user_id에 'jy'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

print(type(ratings))

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

<class 'pandas.core.frame.DataFrame'>


,user_id,movie_id,title,genre,counts,timestamp
829302,6040,1090,platoon (1986),drama|war,3,956715518
829303,6040,1094,"crying game, the (1992)",drama|romance|war,5,956704887
829304,6040,562,welcome to the dollhouse (1995),comedy|drama,5,956704746
829305,6040,1096,sophie's choice (1982),drama,4,956715648
829306,6040,1097,e.t. the extra-terrestrial (1982),children's|drama|fantasy|sci-fi,4,956715569
490346,jy,1097,e.t. the extra-terrestrial (1982),children's|drama|fantasy|sci-fi,5,966570635
655752,jy,3684,"fabulous baker boys, the (1989)",drama|romance,5,963534000
396022,jy,3148,"cider house rules, the (1999)",drama,5,971120805
279761,jy,1213,goodfellas (1990),crime|drama,5,1020387314
617895,jy,3100,"river runs through it, a (1992)",drama,5,965067173


<br>

#### (5) 모델에 활용하기 위한 전처리
---


In [512]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트, indexing 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k, v in enumerate(user_unique)}
movie_to_idx = {v:k for k, v in enumerate(movie_unique)}

In [513]:
# 인덱싱 확인

print(user_to_idx['jy'])
print(movie_to_idx['manhattan murder mystery (1993)'])

6039
1579


In [514]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings.tail(10)

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,title,genre,counts,timestamp
829302,6038,1090,1022,drama|war,3,956715518
829303,6038,1094,980,drama|romance|war,5,956704887
829304,6038,562,311,comedy|drama,5,956704746
829305,6038,1096,142,drama,4,956715648
829306,6038,1097,26,children's|drama|fantasy|sci-fi,4,956715569
490346,6039,1097,26,children's|drama|fantasy|sci-fi,5,966570635
655752,6039,3684,1247,drama|romance,5,963534000
396022,6039,3148,391,drama,5,971120805
279761,6039,1213,269,crime|drama,5,1020387314
617895,6039,3100,554,drama,5,965067173


<br>

## Step 2. CSR Matrix를 직접 만들어보자
---

**CSR Matrix란?**

- Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 ***데이터의 값***과 ***좌표 정보***만으로 구성하여 메모리 사용량을 최소화하면서도 sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조이다.


- data, indices, indptr로 행렬을 압축하여 표현한다.

![image](https://user-images.githubusercontent.com/103712369/175931541-f15af676-b5cf-4fe5-97b9-15b7cf427fc7.png)


- **data**는 0이 아닌 원소를 차례로 기입한 값

> data = [1,2,3,4,5,6]


- **indices**는 data의 각 요소가 어느 열(column)에 있는지를 표현한 index이다.

> indices = [0,4,1,3,0,3]


- **indptr**은 [최초시작행번호, 시작행에서의 데이터 개수, 두번째 행에서의 데이터 누적 개수, ..., 마지막행에서의 데이터 누적개수]이다. 이를 통해 data의 요소들이 어느 행(row)에 있는지를 알 수 있다.

> indptr=[0,2,4,4,6]


- 이를 통해 data[0:2]는 첫번째 행, data[2:4]는 두번째 행, data[4:4]는 세번째 행, data[4:6]는 네번째 행에 위치함을 알 수 있다.

In [515]:
ratings.shape

(829312, 6)

In [516]:
# 실습 위에 설명보고 이해해서 만들어보기

from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie_id = ratings['title'].nunique()

print(num_user, num_movie_id)

6040 3561


In [517]:
print(len(ratings.counts))
print(len(ratings.user_id))
print(len(ratings.movie_id))

829312
829312
829312


In [518]:
print(ratings['user_id'].nunique())
print(ratings['title'].nunique())

6040
3561


In [520]:
ratings.user_id.max()

6039

In [519]:
ratings.title.max()

3560

In [522]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape=(num_user, num_movie_id))
csr_data

<6040x3561 sparse matrix of type '<class 'numpy.int64'>'
	with 829312 stored elements in Compressed Sparse Row format>

<br>

## Step 3. MF 모델 학습하기
---

***`implicit`패키지란?***

- 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지


- implicit 패키지에 구현된 `als(AlternatingLeastSquares) 모델`을 사용한다.


- `Matrix Factorization`에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적이다.

In [523]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습내용과는 무관함!!!
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

***AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보면...***

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지(K)
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지

In [524]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [525]:
# als모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose를 해준다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<3561x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 829312 stored elements in Compressed Sparse Column format>

In [526]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

<br>

## Step 4. 훈련된 모델 선호도 파악
---

In [527]:
jy, goodfellas = user_to_idx['jy'], movie_to_idx['goodfellas (1990)']
jy_vector, goodfellas_vector = als_model.user_factors[jy], als_model.item_factors[goodfellas]

In [528]:
print(len(jy_vector), type(jy_vector))

print(jy_vector.shape)

print(jy_vector)

100 <class 'numpy.ndarray'>
(100,)
[ 0.22186042 -1.0887425  -0.56041574 -0.33027467  0.39505953 -0.03051754
  0.31310356  0.13300717 -0.52867836  0.37170014  0.02832183 -0.29034427
  0.48032647 -0.35747743 -0.5892111   0.93272585 -0.36974669 -0.11099128
 -1.0421591  -0.69918764  0.6145991  -0.4497033   0.7882556  -0.5236878
 -0.11189041  0.02077426 -0.18421473 -0.7535732   0.26780963  1.401979
  0.6817485   0.17132099 -0.320341    1.1272773  -0.48687014  0.41911492
  0.44228876  0.35248414  0.6673535   0.00537635 -0.1096393  -0.14608546
 -0.24016015 -0.3989095  -0.26575813  0.07017139  1.2514086   0.66447705
  0.5075714   0.22248217 -0.1359616   0.27809802  0.15878658  0.13397196
  0.03924403 -0.4548568  -0.5853976  -0.44623175 -0.11422486  0.6982573
  0.6062568   0.14816882  0.26802605  0.20125027  0.30593807 -0.3240363
  0.25041056  0.12336278  0.08242155 -0.41720158 -0.6543189  -0.45017266
  0.2544328   0.55550975 -0.05749025  0.26979288  0.1792255   0.46511626
 -0.17203563 -0.31374

In [529]:
print(goodfellas_vector.shape, type(goodfellas_vector))

print(goodfellas_vector)

(100,) <class 'numpy.ndarray'>
[ 1.38148945e-02 -8.79161432e-03  1.45877227e-02 -1.50724724e-02
 -9.12345853e-03 -4.65839030e-03  4.56194691e-02 -8.46699066e-03
  1.67946459e-03 -2.03100941e-03  3.68488207e-03 -2.18006968e-02
  1.46182943e-02 -2.23569851e-02  4.37625078e-03  1.39450673e-02
 -4.81581781e-03  1.28757348e-02 -3.15655209e-02 -5.85485762e-03
 -1.21726878e-02 -4.91444487e-03  1.71360765e-02  2.42205616e-03
  1.75181124e-02  9.99440998e-03 -9.35318321e-03 -1.29789440e-02
  1.03503757e-03  2.56175250e-02 -2.47836491e-04  3.15885898e-03
  1.98854902e-03  9.37172771e-03 -3.83710116e-02  4.39285226e-02
  1.01721976e-02  2.90111639e-03  4.23059752e-03  1.17970509e-02
  2.20211670e-02  1.92610193e-02 -2.45263521e-02 -2.40899958e-02
  3.40062845e-03  1.73254323e-03  2.21877638e-02 -8.16799141e-03
  3.64386290e-02 -3.44913267e-03 -6.85470272e-03  6.35354640e-03
  1.60307102e-02  6.17501000e-03  1.77266747e-02 -7.72448548e-04
 -1.76256162e-03 -6.01521647e-03  1.14759272e-02  2.4408431

In [530]:
# jy_vector와 goodfellas_vector 내적하는 코드
np.dot(jy_vector, goodfellas_vector)

0.3472094

In [533]:
sophie = movie_to_idx["sophie's choice (1982)"]
sophie_vector = als_model.item_factors[sophie]
np.dot(jy_vector, sophie_vector)

0.10354173

<br>

## Step 5. 비슷한 영화 찾기 + 유저에게 추천하기
---

#### (1) 비슷한 영화 찾기
---

***`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 매서드를 통하여 비슷한 아티스트를 찾는다.***

In [534]:
favorite_movie = 'sixth sense, the (1999)'
movie_idx = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_idx, N=15)
similar_movie

[(38, 0.99999994),
 (121, 0.4620352),
 (273, 0.4519563),
 (233, 0.44336307),
 (243, 0.42422825),
 (235, 0.42210403),
 (828, 0.41564143),
 (792, 0.40899384),
 (81, 0.4031055),
 (170, 0.3980252),
 (220, 0.39729366),
 (3524, 0.38460097),
 (1680, 0.3812162),
 (141, 0.37506574),
 (1917, 0.36696815)]

(아티스트의 id, 유사도) Tuple로 반환하고 있다. 따라서 아티스트의 id를 다시 아티스트의 이름으로 매핑이 필요함.

In [536]:
# artist_to_idx를 뒤집어, index로부터 movie의 이름을 얻는 dict를 생성한다.

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['sixth sense, the (1999)',
 'silence of the lambs, the (1991)',
 'fight club (1999)',
 'usual suspects, the (1995)',
 'ghostbusters (1984)',
 'sleepy hollow (1999)',
 'jakob the liar (1999)',
 'bone collector, the (1999)',
 'green mile, the (1999)',
 'being john malkovich (1999)',
 'seven (se7en) (1995)',
 "heaven's burning (1997)",
 'messenger: the story of joan of arc, the (1999)',
 'fugitive, the (1993)',
 'superstar (1999)']

In [537]:
# 함수로 만들기

def get_similar_movie(movie_name:str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [538]:
get_similar_movie('forrest gump (1994)')

['forrest gump (1994)',
 'groundhog day (1993)',
 'pretty woman (1990)',
 'ghost (1990)',
 'sleepless in seattle (1993)',
 'pleasantville (1998)',
 'clueless (1995)',
 "you've got mail (1998)",
 'four weddings and a funeral (1994)',
 'notting hill (1999)']

In [539]:
get_similar_movie('terminator, the (1984)')

['terminator, the (1984)',
 'aliens (1986)',
 'alien (1979)',
 'die hard (1988)',
 'predator (1987)',
 'terminator 2: judgment day (1991)',
 'matrix, the (1999)',
 'robocop (1987)',
 'blade runner (1982)',
 'mad max 2 (a.k.a. the road warrior) (1981)']

#### (2) 유저에게 영화 추천하기
---

- `AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 매서드를 통하여 제가 좋아할 만한 아티스트를 추천받는다. 


- `filter_already_liked_items`는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [541]:
user = user_to_idx['jy']

# recommend에서는 user*item CSR Matrix를 받는다
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3, 0.41600877),
 (51, 0.31803107),
 (222, 0.28042164),
 (384, 0.25356895),
 (157, 0.24682987),
 (449, 0.23199388),
 (742, 0.22574386),
 (460, 0.22462031),
 (813, 0.21084957),
 (248, 0.2077308),
 (980, 0.19895054),
 (605, 0.19745994),
 (99, 0.1954073),
 (1497, 0.19297987),
 (467, 0.19105008),
 (121, 0.18829589),
 (273, 0.18721741),
 (231, 0.18535917),
 (317, 0.17864366),
 (885, 0.16678819)]

In [542]:
[idx_to_movie[i[0]] for i in movie_recommended]

['erin brockovich (2000)',
 'fargo (1996)',
 'pulp fiction (1994)',
 'jerry maguire (1996)',
 'shawshank redemption, the (1994)',
 'leaving las vegas (1995)',
 'talented mr. ripley, the (1999)',
 'boiler room (2000)',
 'sneakers (1992)',
 'good will hunting (1997)',
 'crying game, the (1992)',
 'godfather, the (1972)',
 'american beauty (1999)',
 'english patient, the (1996)',
 'magnolia (1999)',
 'silence of the lambs, the (1991)',
 'fight club (1999)',
 'insider, the (1999)',
 'twelve monkeys (1995)',
 'professional, the (a.k.a. leon: the professional) (1994)']

<br>

- `good will hunting (1997)`를 추천해주고 있다. 이유를 알고싶다면 `AlternatingLeastSquares` 클래스에 구현된 `explain` 매서드를 사용하면 기록에 남은 데이터 중 이 추천에 기여한 정도를 확이할 수 있다.

In [543]:
good_will_hunting = movie_to_idx['good will hunting (1997)']
explain = als_model.explain(user, csr_data, itemid=good_will_hunting)

위 explain 매서드는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도를 반환한다.

In [544]:
explain

(0.2047465093653938,
 [(269, 0.1172479159634382),
  (554, 0.08699775761057085),
  (391, 0.005667267785484938),
  (26, 0.005556262645307962),
  (1247, -0.010722694639408172)],
 (array([[ 0.59430613,  0.06571731,  0.10366294, ...,  0.12250488,
           0.12275017,  0.08400491],
         [ 0.0390562 ,  0.57109154,  0.112381  , ...,  0.09968391,
           0.07251656,  0.09685772],
         [ 0.06160752,  0.07099229,  0.59955775, ...,  0.0552488 ,
           0.05331151,  0.08836333],
         ...,
         [ 0.0728054 ,  0.06497933,  0.05702664, ...,  0.51055096,
           0.02479781, -0.00371328],
         [ 0.07295118,  0.0494804 ,  0.05283745, ...,  0.07719431,
           0.52864165,  0.02018003],
         [ 0.04992463,  0.0608352 ,  0.07257208, ...,  0.05488046,
           0.06789586,  0.50954208]]),
  False))

In [545]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('goodfellas (1990)', 0.1172479159634382),
 ('river runs through it, a (1992)', 0.08699775761057085),
 ('cider house rules, the (1999)', 0.005667267785484938),
 ('e.t. the extra-terrestrial (1982)', 0.005556262645307962),
 ('fabulous baker boys, the (1989)', -0.010722694639408172)]

\>> `goodfellas (1990)`와 `river runs through it, a (1992)`가 가장 크게 기여했다.

<br>

## 결론 및 회고
---

- Movielens 영화 추천 실습을 진행해보았다. 


- 이번 프로젝트에서는 전처리 작업을 수행하는데 pandas를 주로 사용하였다. pandas의 매서드를 찾아가면서 공부하느라 시간은 오래 걸렸지만 pandas 사용하는데 있어 조금 더 익숙해진 것 같다.


- 내가 선호하는 5가지 영화중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보았다. 결과는 아래와 같다. 

![image](https://user-images.githubusercontent.com/103712369/176018407-17fbc1d6-39eb-44b5-a905-df167c993a3b.png)


- 생각보다 내가 좋아할 영화와 비슷한 영화를 추천해주는 솔루션은 비교적 잘 작동하는 것 같다.

![image](https://user-images.githubusercontent.com/103712369/176018720-c175d742-e370-44a4-a8fe-5820c55ad11f.png)